# SARIMAX - Análise de Produção Solar na França

## Objetivo
Modelar e prever a produção de energia solar na França usando SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors)

## Dataset
- **Arquivo**: solar_france.xlsx
- **Variável Target**: Production (produção de energia solar)
- **Frequência**: Dados horários (2020)
- **Tipo**: Série temporal univariada com sazonalidade diária

## Parâmetros SARIMAX
- **order**: (1, 1, 1)
- **seasonal_order**: (2, 1, 1, 24)

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Análise de séries temporais
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Transformações
from scipy.stats import boxcox
from scipy.special import inv_boxcox

# Métricas de avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import json

# Configurações de visualização
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

# Criar diretório de saída
output_dir = '../../out/solar_france/SARIMAX'
os.makedirs(output_dir, exist_ok=True)

## 1. Carregamento e Exploração Inicial dos Dados

In [ ]:
# Carregamento dos dados
data_path = '../../data/solar_france.xlsx'
df = pd.read_excel(data_path)

print("Informações básicas do dataset:")
print(f"Shape: {df.shape}")
print(f"\nTipos de dados:\n{df.dtypes}")
print(f"\nPrimeiras linhas:")
df.head(10)

In [ ]:
# Preparação da série temporal
df['Date and Hour'] = pd.to_datetime(df['Date and Hour'])
df = df.set_index('Date and Hour')
df = df.sort_index()

# Remover valores nulos se houver
print(f"Valores nulos: {df['Production'].isnull().sum()}")
df = df.dropna()

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(df['Production'].describe())

# Verificar frequência dos dados
freq = pd.infer_freq(df.index)
print(f"\nFrequência inferida: {freq}")
print(f"Período total: {df.index.min()} a {df.index.max()}")
print(f"Duração: {df.index.max() - df.index.min()}")

## 2. Análise Exploratória e Visualização

In [ ]:
# Visualização da série temporal
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Série original
axes[0,0].plot(df.index, df['Production'], alpha=0.7)
axes[0,0].set_title('Série Temporal Original - Produção Solar')
axes[0,0].set_ylabel('Produção (MW)')

# Histograma
axes[0,1].hist(df['Production'], bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Distribuição da Produção')
axes[0,1].set_xlabel('Produção (MW)')
axes[0,1].set_ylabel('Frequência')

# Box plot
axes[1,0].boxplot(df['Production'])
axes[1,0].set_title('Box Plot da Produção')
axes[1,0].set_ylabel('Produção (MW)')

# Q-Q plot
stats.probplot(df['Production'], dist="norm", plot=axes[1,1])
axes[1,1].set_title('Q-Q Plot')

plt.tight_layout()
plt.savefig(f'{output_dir}/exploratory_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Análise por hora do dia (padrão diário)
df['hour'] = df.index.hour
hourly_stats = df.groupby('hour')['Production'].agg(['mean', 'std', 'min', 'max'])

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(hourly_stats.index, hourly_stats['mean'], marker='o', label='Média')
ax.fill_between(hourly_stats.index, 
                hourly_stats['mean'] - hourly_stats['std'],
                hourly_stats['mean'] + hourly_stats['std'],
                alpha=0.3, label='±1 Desvio Padrão')
ax.set_xlabel('Hora do Dia')
ax.set_ylabel('Produção (MW)')
ax.set_title('Padrão Diário de Produção Solar')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig(f'{output_dir}/daily_pattern.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nEstatísticas por hora:")
print(hourly_stats)

## 3. Decomposição da Série Temporal

In [ ]:
# Decomposição sazonal (período de 24 horas)
decomposition = seasonal_decompose(df['Production'], model='additive', period=24)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='Série Original')
decomposition.trend.plot(ax=axes[1], title='Tendência')
decomposition.seasonal.plot(ax=axes[2], title='Componente Sazonal (24h)')
decomposition.resid.plot(ax=axes[3], title='Resíduos')

plt.tight_layout()
plt.savefig(f'{output_dir}/decomposition.png', dpi=300, bbox_inches='tight')
plt.show()

# Análise dos componentes
print("\nVariância dos componentes:")
print(f"Série original: {decomposition.observed.var():.2f}")
print(f"Tendência: {decomposition.trend.dropna().var():.2f}")
print(f"Sazonal: {decomposition.seasonal.var():.2f}")
print(f"Resíduos: {decomposition.resid.dropna().var():.2f}")

## 4. Testes de Estacionariedade

In [ ]:
def test_stationarity(series, name="Série"):
    """
    Testa estacionariedade usando ADF e KPSS
    """
    print(f"\n{'='*50}")
    print(f"Testes de Estacionariedade - {name}")
    print(f"{'='*50}")
    
    # Teste ADF (H0: série não é estacionária)
    adf_result = adfuller(series.dropna())
    print("\nTeste ADF (Augmented Dickey-Fuller):")
    print(f"Estatística de teste: {adf_result[0]:.4f}")
    print(f"P-valor: {adf_result[1]:.4f}")
    print(f"Valores críticos:")
    for key, value in adf_result[4].items():
        print(f"  {key}: {value:.4f}")
    
    if adf_result[1] < 0.05:
        print("✓ Série é estacionária (ADF)")
    else:
        print("✗ Série NÃO é estacionária (ADF)")
    
    # Teste KPSS (H0: série é estacionária)
    kpss_result = kpss(series.dropna())
    print("\nTeste KPSS:")
    print(f"Estatística de teste: {kpss_result[0]:.4f}")
    print(f"P-valor: {kpss_result[1]:.4f}")
    print(f"Valores críticos:")
    for key, value in kpss_result[3].items():
        print(f"  {key}: {value:.4f}")
    
    if kpss_result[1] > 0.05:
        print("✓ Série é estacionária (KPSS)")
    else:
        print("✗ Série NÃO é estacionária (KPSS)")

# Testar série original
test_stationarity(df['Production'], "Produção Solar Original")

# Testar primeira diferença
df['Production_diff'] = df['Production'].diff()
test_stationarity(df['Production_diff'], "Primeira Diferença")

# Testar diferença sazonal (24 horas)
df['Production_seasonal_diff'] = df['Production'].diff(24)
test_stationarity(df['Production_seasonal_diff'], "Diferença Sazonal (24h)")

## 5. ACF e PACF

In [ ]:
# Plotar ACF e PACF
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Série original
plot_acf(df['Production'].dropna(), lags=72, ax=axes[0,0])
axes[0,0].set_title('ACF - Série Original')
plot_pacf(df['Production'].dropna(), lags=72, ax=axes[0,1])
axes[0,1].set_title('PACF - Série Original')

# Primeira diferença
plot_acf(df['Production_diff'].dropna(), lags=72, ax=axes[1,0])
axes[1,0].set_title('ACF - Primeira Diferença')
plot_pacf(df['Production_diff'].dropna(), lags=72, ax=axes[1,1])
axes[1,1].set_title('PACF - Primeira Diferença')

# Diferença sazonal
plot_acf(df['Production_seasonal_diff'].dropna(), lags=72, ax=axes[2,0])
axes[2,0].set_title('ACF - Diferença Sazonal (24h)')
plot_pacf(df['Production_seasonal_diff'].dropna(), lags=72, ax=axes[2,1])
axes[2,1].set_title('PACF - Diferença Sazonal (24h)')

plt.tight_layout()
plt.savefig(f'{output_dir}/acf_pacf.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Divisão dos Dados

In [ ]:
# Divisão treino-teste (80-20)
train_size = int(len(df['Production']) * 0.8)
train = df['Production'].iloc[:train_size]
test = df['Production'].iloc[train_size:]

print(f"Tamanho do treino: {len(train)} observações")
print(f"Tamanho do teste: {len(test)} observações")
print(f"\nPeríodo de treino: {train.index.min()} a {train.index.max()}")
print(f"Período de teste: {test.index.min()} a {test.index.max()}")

# Visualizar divisão
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(train.index, train.values, label='Treino', alpha=0.7)
ax.plot(test.index, test.values, label='Teste', alpha=0.7)
ax.axvline(x=train.index[-1], color='r', linestyle='--', label='Divisão Treino/Teste')
ax.set_xlabel('Data')
ax.set_ylabel('Produção (MW)')
ax.set_title('Divisão Treino-Teste')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{output_dir}/train_test_split.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Modelagem SARIMAX

Usando parâmetros especificados:
- **order**: (1, 1, 1)
- **seasonal_order**: (2, 1, 1, 24)

In [ ]:
# Parâmetros SARIMAX especificados
order = (1, 1, 1)
seasonal_order = (2, 1, 1, 24)

print(f"Treinando modelo SARIMAX{order}x{seasonal_order}...")
print("Isso pode levar alguns minutos...")

# Ajustar modelo
model = SARIMAX(
    train,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_fit = model.fit(disp=False, maxiter=200)

print("\nModelo ajustado com sucesso!")
print(model_fit.summary())

## 8. Previsões

In [ ]:
# Fazer previsões no conjunto de teste
forecast = model_fit.forecast(steps=len(test))
forecast_df = pd.DataFrame({
    'real': test.values,
    'previsto': forecast.values
}, index=test.index)

# Visualizar previsões
fig, ax = plt.subplots(figsize=(15, 6))

# Plotar últimos dias do treino para contexto
ax.plot(train.index[-7*24:], train.values[-7*24:], label='Treino (últimos 7 dias)', alpha=0.7)
ax.plot(test.index, test.values, label='Real', alpha=0.7)
ax.plot(forecast.index, forecast.values, label='Previsto', alpha=0.7)
ax.axvline(x=train.index[-1], color='r', linestyle='--', label='Início do Teste')
ax.set_xlabel('Data')
ax.set_ylabel('Produção (MW)')
ax.set_title('SARIMAX - Previsões vs Real')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{output_dir}/predictions.png', dpi=300, bbox_inches='tight')
plt.show()

# Zoom nos primeiros 3 dias de teste
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(test.index[:72], test.values[:72], label='Real', marker='o', alpha=0.7)
ax.plot(forecast.index[:72], forecast.values[:72], label='Previsto', marker='s', alpha=0.7)
ax.set_xlabel('Data')
ax.set_ylabel('Produção (MW)')
ax.set_title('SARIMAX - Primeiros 3 dias de previsão (zoom)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{output_dir}/predictions_zoom.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Avaliação do Modelo

In [ ]:
# Calcular métricas
mse = mean_squared_error(test, forecast)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test, forecast)
mape = np.mean(np.abs((test - forecast) / (test + 1e-10))) * 100
r2 = r2_score(test, forecast)

# Criar residuos
residuals = test - forecast

print("\n" + "="*50)
print("MÉTRICAS DE AVALIAÇÃO")
print("="*50)
print(f"MSE (Mean Squared Error): {mse:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"R² Score: {r2:.4f}")

# Análise de resíduos
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Resíduos ao longo do tempo
axes[0,0].plot(residuals.index, residuals.values)
axes[0,0].axhline(y=0, color='r', linestyle='--')
axes[0,0].set_title('Resíduos ao Longo do Tempo')
axes[0,0].set_xlabel('Data')
axes[0,0].set_ylabel('Resíduo')
axes[0,0].grid(True, alpha=0.3)

# Histograma dos resíduos
axes[0,1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Distribuição dos Resíduos')
axes[0,1].set_xlabel('Resíduo')
axes[0,1].set_ylabel('Frequência')

# Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot dos Resíduos')

# ACF dos resíduos
plot_acf(residuals, lags=48, ax=axes[1,1])
axes[1,1].set_title('ACF dos Resíduos')

plt.tight_layout()
plt.savefig(f'{output_dir}/residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Teste de Ljung-Box para autocorrelação dos resíduos
lb_test = acorr_ljungbox(residuals, lags=[24, 48], return_df=True)
print("\nTeste de Ljung-Box (autocorrelação dos resíduos):")
print(lb_test)

## 10. Salvar Resultados

In [ ]:
# Salvar previsões
forecast_df.to_csv(f'{output_dir}/sarimax_predictions.csv')
print(f"Previsões salvas em: {output_dir}/sarimax_predictions.csv")

# Salvar métricas e parâmetros
results = {
    'model': 'SARIMAX',
    'dataset': 'solar_france',
    'order': order,
    'seasonal_order': seasonal_order,
    'metrics': {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'mape': float(mape),
        'r2': float(r2)
    },
    'train_size': len(train),
    'test_size': len(test),
    'aic': float(model_fit.aic),
    'bic': float(model_fit.bic)
}

with open(f'{output_dir}/sarimax_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f"Resultados salvos em: {output_dir}/sarimax_results.json")

print("\n✓ Análise SARIMAX concluída com sucesso!")